In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
import xlsxwriter

In [27]:
PATH = r"C:\Users\jamin\OneDrive\Desktop\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get(r"https://www.mlb.com/transactions/")


C:\Users\jamin\AppData\Local\Temp\ipykernel_28640\810215333.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [22]:
#creating a navigation so the starting date and ending dates of the season can be looped through for each year
season_dct = {0:["2011", '2','31','8','28'], 1:["2012",'2', '28', '9', '3'], 2:["2013",'2','31','8','30'],
              3:["2014", '2', '22', '8', '28'], 4:["2015",'3','5','9','4'], 5:["2016",'3','3','9','2'],
              6:["2017",'3','2','9','1'], 7:["2018",'2','29','9','1'], 8:["2019",'2','20','8','29'],
              9:["2020",'6','23','8','27'], 10:["2021",'3','1','9','3'], 11:["2022", '2', '31', '9','5']}
search_paternity = "activated"
search_pat = "from the paternity list"
players_dct = {}
global start_year
start_year = 0


In [23]:
#helper function to find string of player
def find_name(page):
    batter_name = driver.find_elements(By.CLASS_NAME, "player-link")
    transaction_detail = driver.find_elements(By.CLASS_NAME, "description")
    batter_index = 0
    attempts = 0;
    while(attempts < 2):
        try: 
            driver.find_elements(By.CLASS_NAME)
            current_day = driver.find_elements(By.CLASS_NAME, "date")
            break
        except StaleElementException:
            print("attempt1")
        attempts = attempts + 1 
    day_get = current_day[1].text
    import_day = "20" + day_get[6:8] + "-" + day_get[0:2] + "-" + day_get[3:5]
    for i in range(len(batter_name)):
        #batter indexing helps when a player's name isn't in a class so the indexing is thrown off
        if batter_name[batter_index].text not in transaction_detail[i].text:
            batter_index = batter_index - 1
            
        if search_pat in transaction_detail[i].text and search_paternity in transaction_detail[i].text:
            hitter_name = batter_name[batter_index].text
            players_dct[hitter_name] = import_day
        batter_index = batter_index + 1
#helper function that checks if there's multiple pages of transactions on a given day
def check_next_page():
    temp = driver.find_elements("xpath",r"//a[normalize-space()='Next »']")
    if len(temp) == 0:
        return False
    else:
        return True

In [32]:
#function that will iterate through the boundaries of a year
def search_dates(year, month):
    global start_year
    #Initial code outside of for loop is to get to the date of the first year.
    
    #driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
    #Edge case where 2011 isn't in the initial drop down menu, 2014 had to be selected first because then 2011 will show up.
    time.sleep(2)
    year_new = Select(driver.find_element("xpath", '/html/body/main/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div/div[2]/select'))
    year_new.select_by_value(season_dct[3][0])
    time.sleep(2)
    year_new.select_by_value(season_dct[0][0])
    driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
    
    #For loop that iterates through each year
    for i in range(start_year, len(season_dct)):
        #Code outside of the first while loop is to get the correct day and year the MLB season started.
        driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
        time.sleep(2)
        year_inside = Select(driver.find_element("xpath", '/html/body/main/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div/div[2]/select'))
        #Continuation of the edge case where 2011 isn't in the initial drop down menu, if isn't 
        #necessarily needed unless the driver doesn't run at a good speed.
        if(season_dct[i][0] != '2011'):
            driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
            time.sleep(2)
            year_inside.select_by_value(season_dct[i][0])
        time.sleep(2)
        driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
        month_new = Select(driver.find_element("xpath", '/html/body/main/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/select'))
        time.sleep(2)
        month_new.select_by_value(season_dct[i][1])
        time.sleep(2)
        driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
        all_rows = driver.find_elements("xpath", r"//div[@class='pika-lendar']//tbody//td[@class!='is-empty']")
        date_start = season_dct[i][2]
        for date in all_rows:
            if date.get_attribute("data-day") == date_start:
                time.sleep(2)                
                date.click()
                break
        time.sleep(3)
        current_day = driver.find_elements(By.CLASS_NAME, "date")
        day_get = current_day[1].text
        #Checks if the current month is in the season boundaries.
        while((int(day_get[0:2]) <= (int(season_dct[i][3]) + 1))):
            next_day = False
            #Checks if the both day and month are outside of the season boundaries.
            if((int(day_get[3:5]) > int(season_dct[i][4])) and (int(day_get[0:2]) >= (int(season_dct[i][3]) + 1))):
                break;
            attempts = 0;
            while(attempts < 2):
                try: 
                    driver.find_elements(By.CLASS_NAME)
                    current_day = driver.find_elements(By.CLASS_NAME, "date")
                    break
                except StaleElementReferenceException:
                    time.sleep(1)
                attempts = attempts + 1 
            temp = driver.find_elements(By.CLASS_NAME,'noContent')
            if(len(temp) == 1):
                driver.find_element("xpath","//button[@aria-label='Next Day']").click()
                time.sleep(1)
                temp = 0
                continue
            day_get = current_day[1].text
            time.sleep(1)
            import_day = "20" + day_get[6:8] + "-" + day_get[0:2] + "-" + day_get[3:5]
            time.sleep(1)
            #Checking if the key words are on the page.
            if search_pat and search_paternity in driver.page_source:
                find_name(driver.page_source)
            next_day = False
            #Next Day is a boolean that makes the function go back to the top of the while loop if true
            while check_next_page():
                driver.find_element("xpath",r"//a[normalize-space()='Next »']").click()
                next_day = True
                break
            if next_day:
                continue
            attempts2 = 0
            while(attempts2 < 2):
                try:
                    driver.find_element("xpath","//button[@aria-label='Next Day']").click()
                    break
                except NoSuchElementException:
                    driver.refresh()
                    time.sleep(2)
                attempts2 = attempts2 + 1
        start_year += 1
        print("End of Year")

In [31]:
driver.find_element("xpath",'//*[@id="date-container"]/div/div[1]/div[2]/button').click()
time.sleep(2)
year = Select(driver.find_element("xpath",'/html/body/main/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div/div[2]/select'))
month = Select(driver.find_element("xpath", '/html/body/main/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/select'))
search_dates(year, month)

End of Year
End of Year
End of Year
End of Year
End of Year
End of Year


KeyboardInterrupt: 

In [158]:
driver.close()

In [ ]:
#BEGINNING OF FANGRAPH

In [37]:
final_players_sorted['Shohei Ohtani'] = '2022-06-07'

In [87]:
fangraph = webdriver.Chrome(PATH)
fangraph.get(r"https://www.fangraphs.com/")

C:\Users\jamin\AppData\Local\Temp\ipykernel_28640\421894694.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  fangraph = webdriver.Chrome(PATH)


In [73]:
#final_players_sorted = {}
sorted_keys = sorted(players_dct, key=players_dct.get)
#Sort the keys so we can group the stats by year easier.
for w in sorted_keys:
    final_players_sorted[w] = final_players_dct[w]
    
#Values to help us iterate through
player_keys = list(final_players_sorted)
date_keys = list(final_players_sorted.values())

In [58]:
#Lists of stats to be stored in dataframe later
hitters_hr = []
hitters_2b = []
hitters_pa = []
hitters_1b = []
hitters_3b = []
hitters_ob = []
pitchers_ra = []
pitchers_ip = []
pitchers_ha = []
pitchers_bba = []
pitchers_hra = []
pitchers_so = []

In [35]:
#Helper function that finds adds the ten days to find stats.
def second_date(date_key):
    second_day = date_key[0:4] + "-"
    #If the day is above 20, we have to account for month values being updated.
    if(int(date_key[8:10]) > 20):
        #To check for if we have to parse a 0 in front of the month value or not.
        if(int(date_key[5:7]) + 1 < 10):
            new_day = second_day + "0" + str(int(date_key[5:7]) + 1) + "-0" + str((int(date_key[8:10]) + 10) % 10)
        else:
            new_day = second_day + str(int(date_key[5:7])) + "-0" + str((int(date_key[8:10]) + 10) % 10)
    else:
        #To check for if we have to parse a 0 in front of the month value or not.
        if((int(date_key[5:7]) < 10)):
            new_day = second_day + "0" + str(int(date_key[5:7])) + "-" + str((int(date_key[8:10]) + 10))
        else:
            new_day = second_day + str(int(date_key[5:7])) + "-" + str((int(date_key[8:10]) + 10))
    return(new_day)

In [60]:
#Function that finds the table values of the most recent games ten days after being activated. Stores the results as a list.
def find_stats():
    attempts = 0
    #Scraping the tabulated data by dividing it up into rows and columns and scraping each row and column.
    while(attempts < 3):
        try :
            rows = 1 + len(fangraph.find_elements("xpath", "//body[1]/div[1]/div[2]/div[3]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/table[1]/tbody[1]/tr"))
            cols = len(fangraph.find_elements("xpath", "//body[1]/div[1]/div[2]/div[3]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/table[1]/tbody[1]/tr[1]/td"))
        except NoSuchElementException:
            fangraph.refresh()
            time.sleep(3)
        attempts = attempts + 1
    temp = 0
    #Dictionary of the dates the players played.
    players_stat = {}
    for r in range(1, rows):
        #Scraping the data and storing it into lists. 
        list_stats = []
        attempts1 = 0
        #Scraping the rows only first to get the rows, one row represents one game.
        while(attempts1 < 3):
            try:
                date_temp = fangraph.find_element("xpath","//body[1]/div[1]/div[2]/div[3]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/table[1]/tbody[1]/tr["+ str(r) +"]/td[1]")
            except NoSuchElementException:
                fangraph.refresh()
                time.sleep(2)
            attempts1 = attempts1 + 1 
        #SCraping the data by row and column to store each row and column value into the list of the row date.
        for c in range(2, cols + 1):
            attempts2 = 0
            while(attempts2 < 3):
                try:
                    value = fangraph.find_element("xpath","//body[1]/div[1]/div[2]/div[3]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/table[1]/tbody[1]/tr["+ str(r) +"]/td[" + str(c) + "]")
                except NoSuchElementException:
                    fangraph.refresh()
                    time.sleep(2)
                attempts2 = attempts2 + 1
            list_stats.append(value.text) 
        players_stat[temp] = list_stats
        temp = temp + 1
    most_recent_game = len(players_stat) - 1
    temp1 = fangraph.find_elements(By.CLASS_NAME,'player-info-box-pos')
    count = 1
    #Count is how many games you want to track after the transaction.
    #While count < x, count = y, y - x = how many games you want to track.
    while count < 2:
        #If player is a pitcher we want to record different stats.
        if (temp1[0].text == "P"):
            #Edge case if a player doesn't have any games in the ten day time span.
            most_recent_game = most_recent_game - 1
            if(most_recent_game == -1):
                count = 200
                break
            pitchers_ra.append(int(players_stat[most_recent_game][12]))
            pitchers_ip.append(float(players_stat[most_recent_game][8]))
            pitchers_ha.append(int(players_stat[most_recent_game][10]))
            pitchers_hra.append(int(players_stat[most_recent_game][13]))
            pitchers_bba.append(int(players_stat[most_recent_game][14]))
            pitchers_so.append(int(players_stat[most_recent_game][15]))
            count = count + 1
            most_recent_game = most_recent_game - 1
        else:
            if(most_recent_game == -1):
                count = 200
                break
            if (int(players_stat[most_recent_game][5]) < 3):
                most_recent_game = most_recent_game - 1
                continue
            else:
                hitters_pa.append(int(players_stat[most_recent_game][5]))
                singles = int(players_stat[most_recent_game][6]) - (int(players_stat[most_recent_game][7]) + int (players_stat[most_recent_game][8]) +
                                                                 int(players_stat[most_recent_game][9]))
                hitters_1b.append(singles)
                hitters_2b.append(int(players_stat[most_recent_game][7]))
                hitters_3b.append(int(players_stat[most_recent_game][8]))
                hitters_hr.append(int(players_stat[most_recent_game][9]))
                on_base = float(players_stat[most_recent_game][22]) * float(players_stat[most_recent_game][5])
                hitters_ob.append(on_base)
                most_recent_game = most_recent_game - 1
                print(count)
                count = count + 1

In [85]:
#Only run this once so it's initialized before the compile_stats function doesn't restart at 0.
player_count = 0

In [80]:
def compile_stats(player_keys, date_keys):
    #Player_count continues where the driver crashes at so you don't have to restart.
    for i in range(player_count,len(player_keys)):
        if(i != 0):
            #Checks if the year data frames. If the years differ, the list of values are stored into a year dataframe and reset.
            if(int(date_keys[i][:4]) != int(date_keys[i-1][:4])):
                last_year = date_keys[i-1][:4]
                obp = sum(hitters_ob) / sum(hitters_pa)
                total_bases = (sum(hitters_1b) + (sum(hitters_2b) * 2) + (sum(hitters_3b) * 3) + (sum(hitters_hr) * 4)) 
                line = pd.DataFrame({"Plate Appearances": sum(hitters_pa), "Singles": sum(hitters_1b), "Doubles": sum(hitters_2b),
                     "Triples": sum(hitters_3b), "Home Runs": sum(hitters_hr), "On Base Percentage": obp,
                     "Total Bases": total_bases, "Innings Pitched":sum(pitchers_ip), "Hits Allowed": sum(pitchers_ha),
                     "Runs Allowed": sum(pitchers_ra), "Home Runs Allowed": sum(pitchers_hra), "Walks Allowed" : sum(pitchers_bba),
                     "Strikeouts":sum(pitchers_so)}, index=[last_year])
                total_years_dfs.append(line)
                hitters_1b.clear(), hitters_2b.clear(), hitters_3b.clear(), hitters_hr.clear(), hitters_ob.clear(), hitters_pa.clear(),
                pitchers_ip.clear(), pitchers_ha.clear(), pitchers_ra.clear(), pitchers_hra.clear(), pitchers_bba.clear(), pitchers_so.clear()
        player_key = player_keys[i]
        date_key = dates_keys[i]
        fangraph.find_element("xpath", r"//input[@placeholder='Player & Blog Search']").click()
        fangraph.find_element("xpath", r"//div[@class='action-button-view-all fgButton']").click()
        fangraph.find_element("xpath", r"//div[@class='search-full-screen']//input[@placeholder='Player & Blog Search']").send_keys(player_key)
        time.sleep(2)
        fangraph.find_element(By.CLASS_NAME, r"player-name").click()
        temp1 = fangraph.find_elements(By.CLASS_NAME,'player-info-box-pos')
        time.sleep(1)
        if len(temp1) == 0:
            actions = ActionChains(fangraph)
            actions.move_by_offset(10, 10).click().perform()
        time.sleep(3)
        fangraph.execute_script("window.scrollTo(0, 400)") 
        fangraph.find_element("xpath", r"/html/body/div[1]/div[2]/div[2]/div/ul/li[3]/a").click()
        time.sleep(1)
        fangraph.execute_script("window.scrollTo(0, 800)")
        attempts1 = 0
        while(attempts1 < 3):
            try:
                fangraph.find_element(By.CSS_SELECTOR,"#startDate").clear()
                fangraph.find_element(By.CSS_SELECTOR,"#startDate").send_keys(date_key)
                fangraph.find_element(By.CSS_SELECTOR,"#endDate").clear()
                fangraph.find_element(By.CSS_SELECTOR,"#endDate").send_keys(second_date(date_key))
            except NoSuchElementException:
                print("noelement1")
                actions = ActionChains(fangraph)
                actions.move_by_offset(0, 0).click().perform()
                time.sleep(1)
            attempts1 = attempts1 + 1
        attempts = 0
        while(attempts < 3):
            try:
                print(attempts)
                fangraph.find_element(By.CLASS_NAME,"fgButton").click()
                break
            except ElementClickInterceptedException:
                print("clickintercept")
                actions = ActionChains(fangraph)
                actions.move_by_offset(0, 0).click().perform()
                time.sleep(1)
            except NoSuchElementException:
                print("noelement")
                actions = ActionChains(fangraph)
                actions.move_by_offset(0, 0).click().perform()
                time.sleep(1)
            attempts = attempts + 1
        time.sleep(1)
        find_stats()
        player_count = player_count + 1
    #Once all players are iterated through
    last_year = date_keys[i-1][:4]
    obp = sum(hitters_ob) / sum(hitters_pa)
    total_bases = (sum(hitters_1b) + (sum(hitters_2b) * 2) + (sum(hitters_3b) * 3) + (sum(hitters_hr) * 4)) 
    line = pd.DataFrame({"Plate Appearances": sum(hitters_pa), "Singles": sum(hitters_1b), "Doubles": sum(hitters_2b),
            "Triples": sum(hitters_3b), "Home Runs": sum(hitters_hr), "On Base Percentage": obp,
            "Total Bases": total_bases, "Innings Pitched":sum(pitchers_ip), "Hits Allowed": sum(pitchers_ha),
            "Runs Allowed": sum(pitchers_ra), "Home Runs Allowed": sum(pitchers_hra), "Walks Allowed" : sum(pitchers_bba),
            "Strikeouts":sum(pitchers_so)}, index=[last_year])
    total_years_dfs.append(line)
    hitters_1b.clear(), hitters_2b.clear(), hitters_3b.clear(), hitters_hr.clear(), hitters_ob.clear(), hitters_pa.clear(),
    pitchers_ip.clear(), pitchers_ha.clear(), pitchers_ra.clear(), pitchers_hra.clear(), pitchers_bba.clear(), pitchers_so.clear()



In [ ]:
try:
    search_stats(player_keys, date_keys, day_count)
except ElementClickInterceptedException:
    actions.move_by_offset(0, 0).click().perform()

In [1320]:
total_years_dfs

[      Plate Appearances  Singles  Doubles  Triples  Home Runs  \
 2011                 65       15        6        1          2   
 
       On Base Percentage  Total Bases  Innings Pitched  Hits Allowed  \
 2011            0.400215           38             52.9            63   
 
       Runs Allowed  Home Runs Allowed  Walks Allowed  Strikeouts  
 2011            44                 17             29          36  ,
       Plate Appearances  Singles  Doubles  Triples  Home Runs  \
 2012                 99       19        4        2          4   
 
       On Base Percentage  Total Bases  Innings Pitched  Hits Allowed  \
 2012            0.324859           49             42.3            46   
 
       Runs Allowed  Home Runs Allowed  Walks Allowed  Strikeouts  
 2012            16                  2             11          33  ,
       Plate Appearances  Singles  Doubles  Triples  Home Runs  \
 2013                 92       25        2        1          1   
 
       On Base Percentage  T

In [1344]:
#Dataframe that is the end product of compiling from each year.
total_df = pd.concat([total_years_dfs[0], total_years_dfs[1], total_years_dfs[2], total_years_dfs[3],
                     total_years_dfs[4], total_years_dfs[5], total_years_dfs[6], total_years_dfs[7],
                     total_years_dfs[8], total_years_dfs[9], total_years_dfs[10],total_years_dfs[11]])
total_df

,Plate Appearances,Singles,Doubles,Triples,Home Runs,On Base Percentage,Total Bases,Innings Pitched,Hits Allowed,Runs Allowed,Home Runs Allowed,Walks Allowed,Strikeouts
2011,65,15,6,1,2,0.400215,38,52.9,63,44,17,29,36
2012,99,19,4,2,4,0.324859,49,42.3,46,16,2,11,33
2013,92,25,2,1,1,0.374989,36,4.3,4,3,0,2,8
2014,89,16,3,1,2,0.290236,33,25.1,30,11,2,5,16
2015,71,9,2,3,0,0.204254,22,63.0,46,24,7,21,59
2016,97,15,4,0,2,0.271454,31,27.2,26,8,3,4,20
2017,103,17,5,0,1,0.255660,31,71.8,53,17,4,30,73
2018,119,10,8,1,2,0.192555,37,35.7,41,24,3,16,38
2019,86,16,3,0,7,0.339128,50,88.2,95,67,27,34,106
2020,48,7,1,0,0,0.173562,9,43.6,47,28,7,23,59


In [1354]:
total_df.to_excel(r"C:\Users\jamin\OneDrive\Desktop\Projects\baseball birthday performances\Table Stats.xlsx",sheet_name= "Table of Fathers", engine='xlsxwriter')